# Interactive Table and Histogram of Roman Amphitheater Capacities

Execute the three cells in order. That will load data and then create an interactive interface to setting which amphitheaters capacities appear in a table and histogram at the bottom of the page. Slide the sliders and click the checkbox to make changes. Display refresh is clunky, but it works.

Note: If you're running this in Binder, you might get warnings about module compatibility. Should be OK to ignore those, or at least was in Summer of 2018.

In [ ]:
from urllib.request import urlopen
from rdflib import Graph, plugin
import pandas as pd
import matplotlib.pyplot as plt

from ipywidgets import widgets,interact
from IPython.display import display, HTML, clear_output

pd.set_option('display.max_colwidth', -1)

In [ ]:
# jsonld+geojson amphitheater data
json_data = urlopen("http://sfsheath.github.io/roman-amphitheaters/roman-amphitheaters.geojson")
g = Graph().parse(data=json_data.read(), format='json-ld')

In [ ]:
# function that handles changes to sliders and creates histogram
def on_value_change(change=1): # change isn't used but will be passewd so there has to be a placeholder
    global low_widget, high_widget, lower_range, upper_range, sort_widget
    
    # provide defaults if values not defined
    try:
        lower_range = low_widget.value
        upper_range = high_widget.value
        sort_order = sort_widget.value
    except NameError:
        lower_range = 2000
        upper_range = 55000
        sort_order = True
        
# The SPARQL Query
    results = g.query("""
PREFIX ramphs: <http://purl.org/roman-amphitheaters/resource/>
PREFIX gj: <https://purl.org/geojson/vocab#>
PREFIX ramphs-p: <http://purl.org/roman-amphitheaters/properties#>
PREFIX dcterms: <http://purl.org/dc/terms/>
SELECT ?title ?capacity ?pleiades WHERE {

?s gj:properties/dcterms:title ?title ;
   gj:properties/ramphs-p:capacity/ramphs-p:quantity ?capacity ;
   gj:properties/ramphs-p:pleiades ?pleiades
   .

FILTER ((?capacity > %s) )
FILTER (?capacity < %s)
}""" % (lower_range, upper_range ) )

    # build pandas dataframe
    titles, caps, pleiades = [],[],[]
    for row in results:
        titles.append(row.title)
        caps.append(int(row.capacity))
        pleiades.append('<a href="{0}" target="_new">{0}hihi</a>'.format(row.pleiades))
        
    ramphs_pd = pd.DataFrame({'title':titles,'capacity':caps,'pleiades':pleiades})
    
    # build/display interface, table, and histogram
    clear_output()
    low_widget = widgets.IntSlider(value=lower_range,
                               min=2000, max=55000,
                               step = 1000,
                               description="Lower limit:",
                               continuous_update=False)
    high_widget = widgets.IntSlider(value=upper_range,
                               min=2000, max=55000,
                               step = 1000,
                               description="Upper limit:",
                               continuous_update=False)
    sort_widget = widgets.Checkbox(description="Sort high to low", value=sort_order)
    low_widget.observe(on_value_change, names='value')
    high_widget.observe(on_value_change, names='value')
    sort_widget.observe(on_value_change, names='value')
    
    display(high_widget,
            low_widget,
            sort_widget,
            HTML(ramphs_pd.sort_values(by='capacity', ascending = not sort_order).head(10).to_html(escape=False)))
    ramphs_pd.hist() # for now, works better outside of call to display()

# call function that gets the ball rolling,
on_value_change()